In [16]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.wrappers import Bidirectional
from embeddings import Embeddings
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.preprocessing import sequence

In [17]:
embedding = Embeddings(100, 3, 1, 4)

In [18]:
word2vec_model = embedding.get_model()
vocab = embedding.get_vocabulary()

In [19]:
word2index = vocab[0]

In [20]:
contexts = ["""Tesla gained experience in telephony and electrical engineering before emigrating to the United States in 1884 to work for Thomas Edison in New York City.
He soon struck out on his own with financial backers, setting up laboratories and companies to develop a range of electrical devices.
His patented AC induction motor and transformer were licensed by George Westinghouse, who also hired Tesla for a short time as a consultant.
His work in the formative years of electric power development was involved in a corporate alternating current/direct current "War of Currents" as well as various patent battles.
""","""Tesla went on to pursue his ideas of wireless lighting and electricity distribution in his high-voltage, high-frequency power experiments in New York and Colorado Springs, and made early (1893) pronouncements on the possibility of wireless communication with his devices. 
          He tried to put these ideas to practical use in an ill-fated attempt at intercontinental wireless transmission, his unfinished Wardenclyffe Tower project.
          In his lab he also conducted a range of experiments with mechanical oscillators/generators, electrical discharge tubes, and early X-ray imaging.
          He also built a wireless controlled boat, one of the first ever exhibited."""]

In [21]:
max_len = max([len(word_tokenize(context)) for context in contexts])
"""tokenized_contexts = [word_tokenize(context) for context in contexts]
indexed_contexts = [[word2index[word] for word in context if word in word2index] for context in tokenized_contexts]"""

'tokenized_contexts = [word_tokenize(context) for context in contexts]\nindexed_contexts = [[word2index[word] for word in context if word in word2index] for context in tokenized_contexts]'

In [22]:
# padded_contexts = sequence.pad_sequences(indexed_contexts)

In [23]:
target = np.zeros(max_len)

In [24]:
question = "In what year did Nikola Tesla emigrate to the United States?"

In [25]:
from itertools import product

In [26]:
context_question = list(product(contexts, [question]))

In [27]:
# tokenized_sent = [word_tokenize(sent) for sent in sent_tokenize(context)]

In [28]:
def get_average_sentence_vector(sentence):
    avg_sent_vec = np.zeros((100))
    count = 0
    for word in sentence:
        if word in word2vec_model.wv.vocab:
            avg_sent_vec += word2vec_model[word]
            count+=1
    avg_sent_vec = np.divide(avg_sent_vec, count)
    return avg_sent_vec

In [29]:
avg_sentences = []
for context, question in context_question:
    avg_sentences.append(get_average_sentence_vector(word_tokenize(context)))

In [30]:
avg_sentences = np.array(avg_sentences)

In [31]:
avg_question_vec = get_average_sentence_vector(word_tokenize(question))

In [32]:
x = np.array(list(product(avg_sentences, [avg_question_vec])))

In [33]:
x = x.reshape((2, 200))

In [41]:
sent_1_answer_start = target
sent_1_answer_start[13] = 1
sent_2_answer_start = target
y_start = np.array([sent_1_answer_start, sent_2_answer_start])

In [45]:
sent_1_answer_end = target
sent_1_answer_end[13] = 1
sent_2_answer_end = target
y_end = np.array([sent_1_answer_end, sent_2_answer_end])

In [46]:
model = Sequential()
model.add(Dense(100, input_shape=(200,)))
model.add(Dense(y_start.shape[1], activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x, y_start, epochs=100)

Epoch 1/100
2/2 [==============================] - 0s - loss: 4.9004 - acc: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s - loss: 4.3850 - acc: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 0s - loss: 3.7963 - acc: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 0s - loss: 3.1811 - acc: 1.0000
Epoch 5/100
2/2 [==============================] - 0s - loss: 2.5618 - acc: 1.0000
Epoch 6/100
2/2 [==============================] - 0s - loss: 1.9593 - acc: 1.0000
Epoch 7/100
2/2 [==============================] - 0s - loss: 1.4020 - acc: 1.0000
Epoch 8/100
2/2 [==============================] - 0s - loss: 0.9256 - acc: 1.0000
Epoch 9/100
2/2 [==============================] - 0s - loss: 0.5621 - acc: 1.0000
Epoch 10/100
2/2 [==============================] - 0s - loss: 0.3190 - acc: 1.0000
Epoch 11/100
2/2 [==============================] - 0s - loss: 0.1744 - acc: 1.0000
Epoch 12/100
2/2 [==============================] - 0s - loss: 0.0948 - a

2/2 [==============================] - 0s - loss: 3.9548e-05 - acc: 1.0000
Epoch 96/100
2/2 [==============================] - 0s - loss: 3.9548e-05 - acc: 1.0000
Epoch 97/100
2/2 [==============================] - 0s - loss: 3.9548e-05 - acc: 1.0000
Epoch 98/100
2/2 [==============================] - 0s - loss: 3.9489e-05 - acc: 1.0000
Epoch 99/100
2/2 [==============================] - 0s - loss: 3.9489e-05 - acc: 1.0000
Epoch 100/100
2/2 [==============================] - 0s - loss: 3.9459e-05 - acc: 1.0000


In [47]:
model_end = Sequential()
model_end.add(Dense(100, input_shape=(200,)))
model_end.add(Dense(y_start.shape[1], activation="softmax"))
model_end.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model_end.fit(x, y_end, epochs=100)

Epoch 1/100
2/2 [==============================] - 0s - loss: 3.9873 - acc: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s - loss: 3.4234 - acc: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 0s - loss: 2.7885 - acc: 1.0000
Epoch 4/100
2/2 [==============================] - 0s - loss: 2.1459 - acc: 1.0000
Epoch 5/100
2/2 [==============================] - 0s - loss: 1.5384 - acc: 1.0000
Epoch 6/100
2/2 [==============================] - 0s - loss: 1.0116 - acc: 1.0000
Epoch 7/100
2/2 [==============================] - 0s - loss: 0.6070 - acc: 1.0000
Epoch 8/100
2/2 [==============================] - 0s - loss: 0.3380 - acc: 1.0000
Epoch 9/100
2/2 [==============================] - 0s - loss: 0.1810 - acc: 1.0000
Epoch 10/100
2/2 [==============================] - 0s - loss: 0.0966 - acc: 1.0000
Epoch 11/100
2/2 [==============================] - 0s - loss: 0.0527 - acc: 1.0000
Epoch 12/100
2/2 [==============================] - 0s - loss: 0.0297 - acc: 

2/2 [==============================] - 0s - loss: 4.4824e-05 - acc: 1.0000
Epoch 96/100
2/2 [==============================] - 0s - loss: 4.4704e-05 - acc: 1.0000
Epoch 97/100
2/2 [==============================] - 0s - loss: 4.4704e-05 - acc: 1.0000
Epoch 98/100
2/2 [==============================] - 0s - loss: 4.4645e-05 - acc: 1.0000
Epoch 99/100
2/2 [==============================] - 0s - loss: 4.4645e-05 - acc: 1.0000
Epoch 100/100
2/2 [==============================] - 0s - loss: 4.4645e-05 - acc: 1.0000


In [ ]:
model.predict(np.array([x[2]]))

In [48]:
model_end.predict(np.array([x[2]]))

IndexError: index 2 is out of bounds for axis 0 with size 2